In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyenchant

In [ ]:
!pip install pyenchant pysastrawi

In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/libr/libreoffice-dictionaries/hunspell-id_6.4.3-1_all.deb
!dpkg -i hunspell-id_6.4.3-1_all.deb

In [ ]:
!apt update && apt install -y enchant libenchant1c2a hunspell hunspell-en-us libhunspell-1.6-0

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
train_df
test_df
# train_df_sample = train_df.sample(n=50000, random_state=1).reset_index()

In [ ]:
# train_df = train_df.rename(columns={"airline_sentiment": "sentiment"})
train_df = train_df[['text', 'sentiment']]
train_df

In [ ]:
test_df = test_df[['text', 'sentiment']]
test_df

In [ ]:
# train_df = train_df.rename(columns={"airline_sentiment": "sentiment"})
train_df = train_df[['text', 'sentiment']]
train_df

# **Text Preprocessing**

In [ ]:
#removing stopwords
import nltk
from nltk.corpus import stopwords
stopwords.words('english')
from nltk.tokenize import word_tokenize
text = "Nick likes to play football, however he is not too fond of tennis."
text_tokens = word_tokenize(text)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
tokens_without_sw

In [ ]:
import nltk
import sys
import spacy
import re
import enchant

from nltk.corpus import stopwords
import string

BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')

nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])

stop = set(stopwords.words('english'))

punc = set(string.punctuation)

def clean_text(text):

    text = text.lower()
    
    text = BAD_SYMBOLS_RE.sub('', text)

    wordList = text.split()

    wordList = ["".join(x for x in word if x not in punc) for word in wordList]    

    wordList = [word for word in wordList if word not in stop]
    
#     eng_dict = enchant.Dict('en')

#     wordList= [word for word in wordList if eng_dict.check(word)]

    reformed_sentence = " ".join(wordList)
    doc = nlp(reformed_sentence)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
train_normalized=train_df.copy()
val_normalized=test_df.copy()
train_normalized['text'] = train_normalized['text'].astype('str').apply(clean_text)
val_normalized['text'] = val_normalized['text'].astype('str').apply(clean_text)

In [ ]:
X_train=train_normalized['text']
y_train=train_normalized['sentiment']

X_test=val_normalized['text']
y_test=val_normalized['sentiment']

In [ ]:
! pip install -U pip
! pip install -U tensorflow
! pip install -U tensorflow_hub

In [ ]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")

BAD_SYMBOLS_RE = re.compile('[^A-Za-z]+')
def cleanUpEmails(txt):
    txt = txt.lower()
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\S*@\S*\s?', '', txt)
    txt = re.sub('\S*.com.\S*', '', txt)
    txt = re.sub('\S*.\S*.com', '', txt)
    txt = re.sub('pm', '', txt)
    txt = BAD_SYMBOLS_RE.sub(' ', txt)
    
    doc = nlp(txt)

    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    return " ".join([word for word in filtered_sentence if len(word)>1])

In [ ]:
test_emails=pd.read_csv('/kaggle/input/email-sentiment-analysis/test.csv')
test_emails=test_emails['email_body']
test_emails

In [ ]:
test_emails = test_emails.astype('str').apply(cleanUpEmails)

In [ ]:
import tensorflow as tf
tok_emails = tf.keras.preprocessing.text.Tokenizer(nb_words=2500)
tok_emails.fit_on_texts(test_emails)
sequences_test_emails = tok_emails.texts_to_sequences(test_emails)
X_test_for_lstm_emails = tf.keras.preprocessing.sequence.pad_sequences(sequences_test_emails,maxlen=25)

In [ ]:
import tensorflow as tf
tok = tf.keras.preprocessing.text.Tokenizer(nb_words=2500)
all_texts=X_train.append(X_test)
tok.fit_on_texts(all_texts)
sequences_train = tok.texts_to_sequences(X_train)
X_train_for_lstm = tf.keras.preprocessing.sequence.pad_sequences(sequences_train,maxlen=25)
sequences_test = tok.texts_to_sequences(X_test)
X_test_for_lstm = tf.keras.preprocessing.sequence.pad_sequences(sequences_test,maxlen=25)

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.values.reshape(-1, 1))

y_train_transformed = enc.transform(y_train.values.reshape(-1, 1))
y_test_transformed = enc.transform(y_test.values.reshape(-1, 1))

In [ ]:
import scipy
y_train_one_hot=scipy.sparse.csr_matrix.toarray(y_train_transformed)
y_test_one_hot=scipy.sparse.csr_matrix.toarray(y_test_transformed)

In [ ]:
y_train_one_hot

In [ ]:
from tensorflow.keras import backend
def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + backend.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + backend.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+backend.epsilon()))

In [ ]:
MAX_NB_WORDS = 4000

EMBEDDING_DIM = 25

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_for_lstm.shape[1]))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=[tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall(),
                   tf.keras.metrics.AUC(num_thresholds=200,curve="ROC",summation_method="interpolation"),'accuracy',f1])

epochs = 15
batch_size = 64

history = model.fit(X_train_for_lstm, y_train_one_hot,
                    epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test_for_lstm,y_test_one_hot))

In [ ]:
result=model.predict(X_test_for_lstm_emails)

In [ ]:
predict = np.argmax(result, axis=1)

In [ ]:
predict